# Data Cleaning: evaldata

This notebook cleans the `evaldata_raw.xlsx` dataset for modeling. Key steps:
1. Load raw data (wide format: one row per student, year-suffixed columns)
2. Pivot to long format (one row per student-year) — makes cleaning simpler
3. Clean gender, remove PII, handle short enrollment, suspensions, target variable
4. Save cleaned data

In [1]:
import pandas as pd
import numpy as np
import re

## 1. Load Raw Data

In [2]:
df_wide = pd.read_excel('../data/evaldata_raw.xlsx')
print(f'Raw shape: {df_wide.shape}')
df_wide.head()

Raw shape: (79460, 122)


,ANON_ID,Birthdate,Gen,Eth_1718,Fluency_1718,SpEd_1718,SiteName_1718,School Address_1718,City_1718,Zip_1718,...,Grade_2324,AttRate_2324,DaysEnr_2324,DaysAbs_2324,Susp_2324,Address_2324,City_2324.1,Zip_2324.1,CurrWeightedTotGPA_2324,SED_2324
0,1,1997-08-21,F,Asian,RFEP,Not Special Ed,Oakland International HS,4521 Webster St,Oakland,94609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1999-10-10,F,Asian,EL,Not Special Ed,Oakland International HS,4521 Webster St,Oakland,94609.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2019-05-09,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.0,0.9278,180.0,13.0,NaN,7559 Hansom Dr,Oakland,94605.0,NaN,Not SED
3,4,2007-07-05,F,African American,EO,Not Special Ed,EnCompass Academy,1025 81st Avenue,Oakland,94621.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2016-01-26,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,0.9556,180.0,8.0,NaN,6912 Broadway Ter,Oakland,94611.0,NaN,Not SED


## 2. Pivot to Long Format

The wide format has 15 fields repeated across 7 years (e.g., `AttRate_1718`, `AttRate_1819`, ...). 
We pivot to long format: one row per student-year, with columns like `year`, `AttRate`, `DaysEnr`, etc.

In [3]:
# Identify year-suffixed columns and their base names
years = ['1718', '1819', '1920', '2021', '2122', '2223', '2324']
year_pattern = re.compile(r'_(' + '|'.join(years) + r')$')

# Base fields that repeat per year
base_fields = ['Eth', 'Fluency', 'SpEd', 'SiteName', 'School Address', 'City', 'Zip',
               'Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'Susp',
               'Address', 'CurrWeightedTotGPA', 'SED']

# Time-invariant columns
id_cols = ['ANON_ID', 'Birthdate', 'Gen']

# Build long dataframe by stacking each year
records = []
for yr in years:
    yr_cols = {f'{base}_{yr}': base for base in base_fields if f'{base}_{yr}' in df_wide.columns}
    subset = df_wide[id_cols + list(yr_cols.keys())].copy()
    subset.rename(columns=yr_cols, inplace=True)
    subset['year'] = yr
    records.append(subset)

df = pd.concat(records, ignore_index=True)
print(f'Long format shape: {df.shape}')
print(f'Columns: {list(df.columns)}')
df.head()

Long format shape: (556220, 19)
Columns: ['ANON_ID', 'Birthdate', 'Gen', 'Eth', 'Fluency', 'SpEd', 'SiteName', 'School Address', 'City', 'Zip', 'Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'Susp', 'Address', 'CurrWeightedTotGPA', 'SED', 'year']


,ANON_ID,Birthdate,Gen,Eth,Fluency,SpEd,SiteName,School Address,City,Zip,Grade,AttRate,DaysEnr,DaysAbs,Susp,Address,CurrWeightedTotGPA,SED,year
0,1,1997-08-21,F,Asian,RFEP,Not Special Ed,Oakland International HS,4521 Webster St,Oakland,94609.0,12.0,0.9722,180.0,5.0,NaN,920 W MACARTHUR BLVD,3.17,Unknown,1718
1,2,1999-10-10,F,Asian,EL,Not Special Ed,Oakland International HS,4521 Webster St,Oakland,94609.0,12.0,0.9278,180.0,13.0,NaN,920 W MACARTHUR BLVD,2.17,Unknown,1718
2,3,2019-05-09,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1718
3,4,2007-07-05,F,African American,EO,Not Special Ed,EnCompass Academy,1025 81st Avenue,Oakland,94621.0,5.0,0.9943,174.0,1.0,NaN,6716 MacArthur Blvd Apt 4,NaN,Unknown,1718
4,5,2016-01-26,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1718


In [4]:
# Verify: students with AttRate data per year should match inspection findings
print('Students with AttRate per year:')
print(df.groupby('year')['AttRate'].apply(lambda x: x.notna().sum()))

Students with AttRate per year:
year
1718    39929
1819    39579
1920    38839
2021    37558
2122    36153
2223    36552
2324    36695
Name: AttRate, dtype: int64


## 3. Clean Gender Column

Inspection found 2 records with lowercase 'm' — fix to 'M'.

In [5]:
print('Before:', df['Gen'].value_counts().to_dict())
df['Gen'] = df['Gen'].replace({'m': 'M'})
print('After:', df['Gen'].value_counts().to_dict())

Before: {'M': 289037, 'F': 266203, 'N': 966, 'm': 14}


After: {'M': 289051, 'F': 266203, 'N': 966}


## 4. Remove PII / Non-Useful Columns

Drop student residential address, city, and school address — these are PII and not useful for modeling. Keep `SiteName` (school name) and `Zip` (needed to join crime/socioeconomic data).

In [6]:
pii_cols = ['Address', 'City', 'School Address']
df.drop(columns=pii_cols, inplace=True)
print(f'Shape after dropping PII: {df.shape}')
print(f'Remaining columns: {list(df.columns)}')

Shape after dropping PII: (556220, 16)
Remaining columns: ['ANON_ID', 'Birthdate', 'Gen', 'Eth', 'Fluency', 'SpEd', 'SiteName', 'Zip', 'Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'Susp', 'CurrWeightedTotGPA', 'SED', 'year']


## 5. Filter Out Students With No Target Year Data

Remove students who have no `AttRate` in 2023-24 — they can't be used for training/evaluation. Students only need 2023-24 data; having prior years is optional (used as features).

In [7]:
# Find students who have AttRate in 2324
students_with_target = df.loc[(df['year'] == '2324') & df['AttRate'].notna(), 'ANON_ID'].unique()
print(f'Students with 2324 AttRate: {len(students_with_target)}')
print(f'Total students before filter: {df["ANON_ID"].nunique()}')

df = df[df['ANON_ID'].isin(students_with_target)].copy()
print(f'Total students after filter: {df["ANON_ID"].nunique()}')
print(f'Shape: {df.shape}')

Students with 2324 AttRate: 36695


Total students before filter: 79460


Total students after filter: 36695
Shape: (256865, 16)


## 6. Handle Short Enrollment

Students with `DaysEnr < 30` in a given year have unreliable attendance rates. Set their `AttRate` to NaN for those year-rows rather than dropping the student entirely.

In [8]:
short_mask = df['DaysEnr'].notna() & (df['DaysEnr'] < 30)
print(f'Student-year rows with DaysEnr < 30: {short_mask.sum()}')
print(f'By year:')
print(df.loc[short_mask].groupby('year').size())

df.loc[short_mask, 'AttRate'] = np.nan

Student-year rows with DaysEnr < 30: 1663
By year:
year
1718    113
1819     87
1920    196
2021    108
2122    185
2223    196
2324    778
dtype: int64


## 7. Clean Suspensions

NaN in `Susp` means no suspensions recorded — replace with 0.

In [9]:
print(f'Susp NaN before: {df["Susp"].isna().sum()}')
# Only fill with 0 where the student was actually enrolled that year (has DaysEnr)
enrolled_mask = df['DaysEnr'].notna()
df.loc[enrolled_mask, 'Susp'] = df.loc[enrolled_mask, 'Susp'].fillna(0)
print(f'Susp NaN after: {df["Susp"].isna().sum()}')
print(f'Suspension distribution (enrolled rows):')
print(df.loc[enrolled_mask, 'Susp'].value_counts().head(10))

Susp NaN before: 252839
Susp NaN after: 105736
Suspension distribution (enrolled rows):
Susp
0.0     147103
1.0       2763
2.0        730
3.0        265
4.0        116
5.0         48
6.0         33
7.0         21
8.0          6
11.0         3
Name: count, dtype: int64


## 8. GPA — Leave As-Is

GPA is structurally missing for elementary students (no GPA assigned). Leave NaN as-is; this will be handled during feature engineering.

In [10]:
gpa_avail = df['CurrWeightedTotGPA'].notna() & enrolled_mask
print(f'Rows with GPA: {gpa_avail.sum()} / {enrolled_mask.sum()} enrolled rows')
print(f'Grade range with GPA: {df.loc[gpa_avail, "Grade"].min()} - {df.loc[gpa_avail, "Grade"].max()}')

Rows with GPA: 55534 / 151095 enrolled rows


Grade range with GPA: 6.0 - 12.0


## 9. Define Target Variable

Binary target: `chronic_absent` = 1 if `AttRate < 0.90`, else 0. Only defined for rows that have a valid AttRate.

In [11]:
df['chronic_absent'] = np.where(df['AttRate'].notna(), (df['AttRate'] < 0.90).astype(int), np.nan)

# Check target distribution for 2324
target_2324 = df.loc[df['year'] == '2324', 'chronic_absent'].dropna()
print(f'Target distribution (2023-24):')
print(f'  Chronic absent: {int(target_2324.sum())} ({target_2324.mean()*100:.1f}%)')
print(f'  Not chronic:    {int((target_2324 == 0).sum())} ({(target_2324 == 0).mean()*100:.1f}%)')

Target distribution (2023-24):
  Chronic absent: 11539 (32.1%)
  Not chronic:    24378 (67.9%)


## 10. Join Crime and Socioeconomic Data

Join external datasets (by zip code and year) to enrich student records with neighborhood-level features.

School year codes map to calendar years: `1718` → 2017, `1819` → 2018, etc.

In [12]:
# Map school year codes to calendar years for joining
year_map = {'1718': 2017, '1819': 2018, '1920': 2019, '2021': 2020,
            '2122': 2021, '2223': 2022, '2324': 2023}
df['calendar_year'] = df['year'].map(year_map)

# Clean Zip to integer string for joining (drop decimals, handle NaN)
df['zip_code'] = df['Zip'].apply(lambda x: str(int(x)) if pd.notna(x) else None)

# Load crime data
crime = pd.read_csv('../data/oakland_crime_by_zip.csv', dtype={'zip_code': str})
print(f'Crime data: {crime.shape}')
print(crime.head())

# Load socioeconomic data
socio = pd.read_csv('../data/oakland_socioeconomic_by_zip.csv', dtype={'zip_code': str})
print(f'\nSocioeconomic data: {socio.shape}')
print(socio.head())

Crime data: (136, 7)
   year zip_code  total_crimes  violent_crimes  property_crimes  drug_crimes  \
0  2017    94601          4749            1325             1434          168   
1  2017    94602          1881             231              673           25   
2  2017    94603         10768            3027             3139          614   
3  2017    94605           560              64              211            2   
4  2017    94606          5037             965             1632          230   

   other_crimes  
0          1822  
1           952  
2          3988  
3           283  
4          2210  

Socioeconomic data: (120, 18)
   year zip_code    area_name  total_population  poverty_universe  \
0  2017    94601  ZCTA5 94601             50763             50201   
1  2017    94602  ZCTA5 94602             30098             29986   
2  2017    94603  ZCTA5 94603             33952             33794   
3  2017    94605  ZCTA5 94605             44260             44024   
4  2017    946

In [13]:
# Join crime data
crime_cols = ['zip_code', 'year', 'total_crimes', 'violent_crimes', 'property_crimes', 'drug_crimes', 'other_crimes']
crime_join = crime[crime_cols].rename(columns={'year': 'calendar_year'})

before = len(df)
df = df.merge(crime_join, on=['zip_code', 'calendar_year'], how='left')
print(f'After crime join: {len(df)} rows (was {before})')
print(f'Crime columns matched: {df["total_crimes"].notna().sum()} / {len(df)} rows')

# Join socioeconomic data — pick useful derived columns, skip raw census counts
socio_cols = ['zip_code', 'year', 'total_population', 'poverty_rate_pct',
              'median_household_income', 'unemployment_rate_pct',
              'high_school_plus_rate_pct', 'college_degree_rate_pct',
              'median_gross_rent', 'median_home_value', 'uninsured_rate_pct']
socio_join = socio[socio_cols].rename(columns={'year': 'calendar_year'})

# Convert numeric columns (they may be strings from CSV)
for col in socio_cols[2:]:
    socio_join[col] = pd.to_numeric(socio_join[col], errors='coerce')

df = df.merge(socio_join, on=['zip_code', 'calendar_year'], how='left')
print(f'After socio join: {len(df)} rows')
print(f'Socio columns matched: {df["median_household_income"].notna().sum()} / {len(df)} rows')

# Drop helper columns used only for joining
df.drop(columns=['calendar_year', 'zip_code'], inplace=True)
print(f'\nFinal columns: {list(df.columns)}')

After crime join: 256865 rows (was 256865)
Crime columns matched: 147607 / 256865 rows
After socio join: 256865 rows
Socio columns matched: 147607 / 256865 rows



Final columns: ['ANON_ID', 'Birthdate', 'Gen', 'Eth', 'Fluency', 'SpEd', 'SiteName', 'Zip', 'Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'Susp', 'CurrWeightedTotGPA', 'SED', 'year', 'chronic_absent', 'total_crimes', 'violent_crimes', 'property_crimes', 'drug_crimes', 'other_crimes', 'total_population', 'poverty_rate_pct', 'median_household_income', 'unemployment_rate_pct', 'high_school_plus_rate_pct', 'college_degree_rate_pct', 'median_gross_rent', 'median_home_value', 'uninsured_rate_pct']


## 11. Final Verification

In [14]:
print(f'Final shape: {df.shape}')
print(f'Unique students: {df["ANON_ID"].nunique()}')
print(f'Years: {sorted(df["year"].unique())}')
print(f'\nColumns: {list(df.columns)}')
print(f'\nDtypes:\n{df.dtypes}')
print(f'\nMissing values:\n{df.isnull().sum()}')
print(f'\nRows per year:')
print(df.groupby('year').size())
print(f'\nAttRate available per year (after short enrollment filter):')
print(df.groupby('year')['AttRate'].apply(lambda x: x.notna().sum()))

Final shape: (256865, 31)
Unique students: 36695
Years: ['1718', '1819', '1920', '2021', '2122', '2223', '2324']

Columns: ['ANON_ID', 'Birthdate', 'Gen', 'Eth', 'Fluency', 'SpEd', 'SiteName', 'Zip', 'Grade', 'AttRate', 'DaysEnr', 'DaysAbs', 'Susp', 'CurrWeightedTotGPA', 'SED', 'year', 'chronic_absent', 'total_crimes', 'violent_crimes', 'property_crimes', 'drug_crimes', 'other_crimes', 'total_population', 'poverty_rate_pct', 'median_household_income', 'unemployment_rate_pct', 'high_school_plus_rate_pct', 'college_degree_rate_pct', 'median_gross_rent', 'median_home_value', 'uninsured_rate_pct']

Dtypes:
ANON_ID                               int64
Birthdate                    datetime64[ns]
Gen                                  object
Eth                                  object
Fluency                              object
SpEd                                 object
SiteName                             object
Zip                                 float64
Grade                               fl

year
1718    36695
1819    36695
1920    36695
2021    36695
2122    36695
2223    36695
2324    36695
dtype: int64

AttRate available per year (after short enrollment filter):


year
1718    11227
1819    13902
1920    16675
2021    19307
2122    23383
2223    29021
2324    35917
Name: AttRate, dtype: int64


## 12. Save Cleaned Data

In [15]:
df.to_csv('../data/evaldata_cleaned.csv', index=False)
print(f'Saved evaldata_cleaned.csv ({df.shape[0]} rows, {df.shape[1]} columns)')

Saved evaldata_cleaned.csv (256865 rows, 31 columns)
